In [151]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import py7zr


In [128]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import os
import torch 
from typing import Any, Dict, List
import pickle

The aim of this notebook is to run a game json file and to produce a cleaned file with all the events. Each event should have trajectories of state-action pairs.

Possible state/action representations:

* 22 dimensional vector for the current state (positions of all 10 players and the ball) - 10 dimension vector for the action (change in x and y for each defensive player)
* 22 dimensional vector for the current state (change in x,y of all 10 players and the ball) - 10 dimension vector for the action (change in x and y for each defensive player)
* 22 dimensional vector for the current state (change in x,y of 5 offensive players and ball, current position of 5 defensive players) - 10 dimension vector for the action (change in x and y for each defensive player)
* 22 dimensional vector for the players (one of above) and 2 dimensional vector of game and shot clock for state - 10 dimension vector for the action (as above)

In [238]:

def start(path_to_json):
    
    data_frame = pd.read_json(path_to_json)
    last_default_index = len(data_frame) - 1

    for event_index in range(last_default_index):
        
        eventnum = data_frame['events'][event_index]
        
        moments = []
        for moment in eventnum['moments']:
            if moment[5][0][0] == -1:
                moments.append(Moment(moment))
            else:
                break

        if len(moments) >= 10:
            save(moments,path_to_json[10:-5],event_index)
            
        

In [239]:
class Moment:
    def __init__(self, moment):
        self.quarter = moment[0]  
        self.game_clock = moment[2]  
        self.shot_clock = moment[3]  
        ball = moment[5][0] 
        self.ball = Ball(ball)
        players = moment[5][1:]  
        self.players = [Player(player) for player in players]

In [240]:
class Player:
    def __init__(self, player):
        self.id = player[1]
        self.x = player[2]
        self.y = player[3]

In [241]:
class Ball:
    def __init__(self, ball):
        self.x = ball[2]
        self.y = ball[3]
        self.z = ball[4]

In [242]:

def makestateaction(moments,i,attacking_team_first):

    if attacking_team_first == True:
        O = 0
        D = 5
    else:
        O = 5
        D = 0

    state = []
    for l in range(5):
        state.append(moments[i].players[O+l].x)
        state.append(moments[i].players[O+l].y)

    for l in range(5):
        state.append(moments[i].players[D+l].x)
        state.append(moments[i].players[D+l].y)

    state.append(moments[i].ball.x)
    state.append(moments[i].ball.y)

    action = []
    for k in range(5):
        action.append(moments[i+1].players[D+k].x - moments[i].players[D+k].x)
        action.append(moments[i+1].players[D+k].x - moments[i].players[D+k].x)

    return torch.as_tensor(state),torch.as_tensor(action),torch.as_tensor(0),torch.as_tensor(False)

def save(moments,gamename,event_index):

    demos = Experiences()
    teams_sum = 0 

    for momentindex in range(len(moments)-1):
        player_to_ball_distance = [np.sqrt((moments[momentindex].players[j].x - moments[momentindex].ball.x)**2 
                                       + (moments[momentindex].players[j].y - moments[momentindex].ball.y)**2) for j in range(10)]
        if np.argmin(player_to_ball_distance) > 4:
            teams_sum += 1

    if teams_sum/(len(moments)-1) < 0.1:
        attacking_team_first = True
        
        for momentindex in range(len(moments)-1):
            state,action,reward,done= makestateaction(moments,momentindex,attacking_team_first)
            demos.add(state,action,reward,done)
            
        with open('PickleDemos/' + gamename + "_" + str(event_index) + ".pkl", 'wb') as f:
            pickle.dump(demos, f, pickle.HIGHEST_PROTOCOL)

    elif teams_sum/(len(moments)-1) > 0.9:
        attacking_team_first = False
        
        for momentindex in range(len(moments)-1):
            state,action,reward,done= makestateaction(moments,momentindex,attacking_team_first)
            demos.add(state,action,reward,done)
            
        with open('PickleDemos/' + gamename + "_" + str(event_index) + ".pkl", 'wb') as f:
            pickle.dump(demos, f, pickle.HIGHEST_PROTOCOL)  

        print(event_index)
        print(teams_sum/(len(moments)-1))
        print(len(moments)-1)

            

In [243]:
class Experiences:
    def __init__(self) -> None:
        """
        Initialize experiences object, which stores a stack of agent-environment 
        transitions.
        """
        self.states = []
        self.actions = []
        self.rewards = []
        self.dones = []

    def add(
        self,
        state: torch.Tensor,
        action: torch.Tensor,
        reward: torch.Tensor,
        done: torch.Tensor,
    ) -> None:
        """
        Add a transition to the stack of transitions.
        Parameters
        ----------
        state: torch.Tensor
        action: torch.Tensor
        reward: torch.Tensor
        done: torch.Tensor
        """
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.dones.append(done)

In [247]:
json_list = os.listdir("JsonFiles/")

for json in json_list:
    print(json)
    json_path = "JsonFiles/" + json
    start(json_path)

0021500001.json
4
1.0
63
12
0.9721706864564007
539
28
1.0
10
31
0.8166666666666667
360


IndexError: list index out of range